# 1 处理prodect_info

In [1]:
import pandas as pd
import numpy as np
import datetime
import gc
import random

In [2]:
product_info = pd.read_csv('./product_info.csv')

C:\Users\Administrator\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', None)
product_info.head()

,order_no,game_id,channel_id,source_id,server_id,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,d_discount,d_money,s_discount,s_money,pay_count,role_pay_count,role_level,device_no,status,pay_time,post_time,active_pay_count,role_name,dt
0,H20200607235949599113164010953,1001081,20814,307611,30860,13273385,113164010,27,1.0,0.0,1.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,10,10,128,F17735C4-5BBE-4E48-9BF8-6F849DDBF263,3,2020-06-08 00:00:09,2020-06-08 00:00:09,10,葛擎苍,2020-06-08
1,H20200608000012775112767396777,1001081,20938,306794,30582,13181677,112767396,27,1.0,0.0,1.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,52,52,163,6A05A10F-CB14-4837-A4EE-B67261C65768,3,2020-06-08 00:00:25,2020-06-08 00:00:25,52,臧渝,2020-06-08
2,H20200608000015629110588172745,1000952,20789,292133,30908,13288577,110588172,2,1.0,0.0,1.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,70,50,0,08D275B5-A684-453B-BE04-CD9D36A84473,3,2020-06-08 00:00:25,2020-06-08 00:00:25,70,乜盼夏,2020-06-08
3,H2020060800001822598005731142,1000952,20769,227280,22646,10183475,98005731,27,6.0,0.0,6.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,106,106,0,19D6756F-6FFD-454A-A607-A40845F26DF9,3,2020-06-08 00:00:32,2020-06-08 00:00:32,62,路安,2020-06-08
4,H20200608000021856104706588285,1001081,20774,272572,26286,11690378,104706588,2,3.0,0.0,3.0,0.0,0.0,NaN,0,0.0,0.0,0.0,0.0,172,152,235,7718269B-BEA5-4143-A5AB-EE0E414E4B03,3,2020-06-08 00:00:45,2020-06-08 00:00:45,152,邓桦,2020-06-08


In [4]:
product_info.shape

(202862, 29)

In [5]:
product_info.drop(['d_discount','d_money', 's_discount', 's_money','status','role_name'], axis=1, inplace=True)

In [6]:
print('本周付费user_id数:',product_info['user_id'].nunique())
print('本周付费设备数:',product_info['device_no'].nunique())
print('本周付费role_id数:',product_info['role_id'].nunique())
print('本周付费rmb:',product_info['rmb_money'].sum())
print('本周平均每单付费rmb:',product_info['rmb_money'].mean())

本周付费user_id数: 41695
本周付费设备数: 39035
本周付费role_id数: 41826
本周付费rmb: 16728002.450000003
本周平均每单付费rmb: 82.46000951385612


In [7]:
# 将所有付费过的user_id标记为1
product_info['is_pay'] = 1

In [8]:
product_info.dtypes

order_no             object
game_id               int64
channel_id            int64
source_id             int64
server_id             int64
role_id               int64
user_id               int64
pay_type              int64
money               float64
give_money          float64
rmb_money           float64
ch_money            float64
coupon_money        float64
coupon_code          object
coupon_type           int64
pay_count             int64
role_pay_count        int64
role_level            int64
device_no            object
pay_time             object
post_time            object
active_pay_count      int64
dt                   object
is_pay                int64
dtype: object

In [9]:
# 按game_id进行分组聚合，统计game本周的付费情况
groups = product_info.groupby(['game_id'])
# game本周总付费次数  p1
temp = groups.size().reset_index().rename(columns={0:'p1'})
matrix_product = pd.merge(product_info, temp, on='game_id', how='left')
# game本周的付费user_id数  p2
temp = groups['user_id'].agg([('p2', 'nunique')]).reset_index()
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
# game本周的付费role_id数  p3
temp = groups['role_id'].agg([('p3', 'nunique')]).reset_index()
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
# game本周的付费device_no数  p4
temp = groups['device_no'].agg([('p4', 'nunique')]).reset_index()
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
# game本周的付费平均每单金额  p5
temp = groups['money'].agg([('p5', 'mean')]).reset_index()
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
# game本周的付费平均代金券金额  p6
temp = groups['coupon_money'].agg([('p6', 'mean')]).reset_index()
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
# game本周的付费平均草花币金额  p7
temp = groups['ch_money'].agg([('p7', 'mean')]).reset_index()
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
matrix_product.head()

,order_no,game_id,channel_id,source_id,server_id,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,pay_count,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt,is_pay,p1,p2,p3,p4,p5,p6,p7
0,H20200607235949599113164010953,1001081,20814,307611,30860,13273385,113164010,27,1.0,0.0,1.0,0.0,0.0,NaN,0,10,10,128,F17735C4-5BBE-4E48-9BF8-6F849DDBF263,2020-06-08 00:00:09,2020-06-08 00:00:09,10,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0
1,H20200608000012775112767396777,1001081,20938,306794,30582,13181677,112767396,27,1.0,0.0,1.0,0.0,0.0,NaN,0,52,52,163,6A05A10F-CB14-4837-A4EE-B67261C65768,2020-06-08 00:00:25,2020-06-08 00:00:25,52,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0
2,H20200608000015629110588172745,1000952,20789,292133,30908,13288577,110588172,2,1.0,0.0,1.0,0.0,0.0,NaN,0,70,50,0,08D275B5-A684-453B-BE04-CD9D36A84473,2020-06-08 00:00:25,2020-06-08 00:00:25,70,2020-06-08,1,12410,1662,1684,1622,77.028445,1.787326,0.0
3,H2020060800001822598005731142,1000952,20769,227280,22646,10183475,98005731,27,6.0,0.0,6.0,0.0,0.0,NaN,0,106,106,0,19D6756F-6FFD-454A-A607-A40845F26DF9,2020-06-08 00:00:32,2020-06-08 00:00:32,62,2020-06-08,1,12410,1662,1684,1622,77.028445,1.787326,0.0
4,H20200608000021856104706588285,1001081,20774,272572,26286,11690378,104706588,2,3.0,0.0,3.0,0.0,0.0,NaN,0,172,152,235,7718269B-BEA5-4143-A5AB-EE0E414E4B03,2020-06-08 00:00:45,2020-06-08 00:00:45,152,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0


In [10]:
def count_1(list):
    count = 0
    for num in list:
        if num ==1:
            count += 1
    return count

In [11]:
# game本周的新增user_id付费人数  p8
temp = groups['pay_count'].apply(count_1).reset_index().rename(columns={'pay_count':'p8'})
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
# game本周的新增role_id付费人数  p9
temp = groups['role_pay_count'].apply(count_1).reset_index().rename(columns={'role_pay_count':'p9'})
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')
# game本周的新增设备付费人数  p10
temp = groups['active_pay_count'].apply(count_1).reset_index().rename(columns={'active_pay_count':'p10'})
matrix_product = pd.merge(matrix_product, temp, on='game_id', how='left')

In [12]:
matrix_product.head()

,order_no,game_id,channel_id,source_id,server_id,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,pay_count,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt,is_pay,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
0,H20200607235949599113164010953,1001081,20814,307611,30860,13273385,113164010,27,1.0,0.0,1.0,0.0,0.0,NaN,0,10,10,128,F17735C4-5BBE-4E48-9BF8-6F849DDBF263,2020-06-08 00:00:09,2020-06-08 00:00:09,10,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0,196,301,594
1,H20200608000012775112767396777,1001081,20938,306794,30582,13181677,112767396,27,1.0,0.0,1.0,0.0,0.0,NaN,0,52,52,163,6A05A10F-CB14-4837-A4EE-B67261C65768,2020-06-08 00:00:25,2020-06-08 00:00:25,52,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0,196,301,594
2,H20200608000015629110588172745,1000952,20789,292133,30908,13288577,110588172,2,1.0,0.0,1.0,0.0,0.0,NaN,0,70,50,0,08D275B5-A684-453B-BE04-CD9D36A84473,2020-06-08 00:00:25,2020-06-08 00:00:25,70,2020-06-08,1,12410,1662,1684,1622,77.028445,1.787326,0.0,210,271,801
3,H2020060800001822598005731142,1000952,20769,227280,22646,10183475,98005731,27,6.0,0.0,6.0,0.0,0.0,NaN,0,106,106,0,19D6756F-6FFD-454A-A607-A40845F26DF9,2020-06-08 00:00:32,2020-06-08 00:00:32,62,2020-06-08,1,12410,1662,1684,1622,77.028445,1.787326,0.0,210,271,801
4,H20200608000021856104706588285,1001081,20774,272572,26286,11690378,104706588,2,3.0,0.0,3.0,0.0,0.0,NaN,0,172,152,235,7718269B-BEA5-4143-A5AB-EE0E414E4B03,2020-06-08 00:00:45,2020-06-08 00:00:45,152,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0,196,301,594


In [13]:
matrix_product.isnull().sum()

order_no                 0
game_id                  0
channel_id               0
source_id                0
server_id                0
role_id                  0
user_id                  0
pay_type                 0
money                    0
give_money               0
rmb_money                0
ch_money                 0
coupon_money             0
coupon_code         152703
coupon_type              0
pay_count                0
role_pay_count           0
role_level               0
device_no             2688
pay_time                 0
post_time                0
active_pay_count         0
dt                       0
is_pay                   0
p1                       0
p2                       0
p3                       0
p4                       0
p5                       0
p6                       0
p7                       0
p8                       0
p9                       0
p10                      0
dtype: int64

In [14]:
# 根据user_id,role_id把缺失的device_no补全，众数补全
def mode_1(x):
    mode_list = x.mode()
    if len(mode_list) ==1:
        mode = mode_list[0]
    if len(mode_list) ==0:
        mode = np.nan
    if len(mode_list)>1:
        mode = random.choice(mode_list)
    return mode
device_no_filler = pd.DataFrame(matrix_product.groupby(['role_id'])['device_no'].apply(mode_1), columns=['device_no'])
matrix_product.update(device_no_filler, overwrite=False)

In [15]:
matrix_product['device_no'].isna().sum()

2688

In [16]:
matrix_product.to_csv('./FE_data/matrix_product.csv', index=None)

In [17]:
del product_info
gc.collect()

60

# 2 处理login_info

In [18]:
login_info = pd.read_csv('./login_info.csv') # pay前一周登录信息

In [19]:
# 压缩数据
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [20]:
login_info = reduce_mem_usage(login_info)

Mem. usage decreased to 1142.50 Mb (25.0% reduction)


In [21]:
login_info.head()

,user_id,role_id,login_time,game_id,role_level,device_no,ip,p_model,sys_ver,net_w,net_type,sdk_no
0,112131331,149481937,2020-06-01 10:20:11,1001049,89,F6FA4ED0B437A712D882FF0A5046E183,111.52.50.96,1605-A01,6.0.1,wifi,1.0,4002000
1,101748615,141238538,2020-06-01 10:19:27,1000840,160,69A334395275B8F38C46D2BAADD31654,111.113.169.37,PAR-AL00,9,4g,3.0,4000000
2,110447841,148183494,2020-06-01 10:20:22,1000748,62,8AF169CB397754A3A9FCF671D8B0545D,183.240.10.135,Mate 10 pro,6.0.1,wifi,0.0,3003002
3,77143990,150540428,2020-06-01 10:19:39,1000571,0,7979847E86528DEBE3322875940579F9,223.88.141.90,V1838A,10,wifi,1.0,4000000
4,104448650,146453280,2020-06-01 10:20:15,1000840,121,09070BB76C84D18F07EF4F501391BE7F,120.235.35.13,V1814A,9,wifi,1.0,4000000


In [22]:
print(login_info.shape)
print('本周有多少不同设备登录:',login_info['device_no'].nunique())
print('本周有多少不同用户登录:',login_info['user_id'].nunique())
print('本周有多少不同角色登录:',login_info['role_id'].nunique())
print('每台设备登录情况统计:')
print(login_info['device_no'].value_counts())


(16638922, 12)
本周有多少不同设备登录: 529784
本周有多少不同用户登录: 711528
本周有多少不同角色登录: 809831
每台设备登录情况统计:
18D9040B22DD5544699A3792EDFA7346        83685
FE5BD6304FB6988C0531F874882BBB85        37176
F8039DB3F1133DE8EAB7C38A7106DEA4        22917
730E9BE3F43442589805CBCCA17BA95C        17150
D361B6CCE3F3B45ECB2E917C970E0CDD        16633
                                        ...  
01604946DA713947A7B9A5A4B44B1063            1
60F28EFABBD1822BA725093C190AB380            1
4965024E94F4D9CEEBE15CD2C6DE2BED            1
4BC753F6D94EDD198415BA33A8E45518            1
B53FF13C-9EBA-4F28-90BA-EB6848E714DA        1
Name: device_no, Length: 529784, dtype: int64


In [23]:
# 按uesr_id进行聚合，查看user_id交互行为
groups = login_info.groupby(['user_id'])
# user本周登录次数 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = pd.merge(login_info, temp, on='user_id', how='left')
# user本周登录的角色数u2
temp = groups['role_id'].agg([('u2', 'nunique')]).reset_index()
matrix = pd.merge(matrix, temp, on='user_id', how='left')
# user本周使用的设备数u3
temp = groups['device_no'].agg([('u3', 'nunique')]).reset_index()
matrix = pd.merge(matrix, temp, on='user_id', how='left')
# user本周使用的ip数u4
temp = groups['ip'].agg([('u4', 'nunique')]).reset_index()
matrix = pd.merge(matrix, temp, on='user_id', how='left')
# 登录的时间间隔u5 按小时
login_info['login_time'] = pd.to_datetime(login_info['login_time'])
temp = groups['login_time'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u5'] = round((temp['L_time']-temp['F_time']).dt.seconds/3600, 2)
matrix = pd.merge(matrix, temp[['user_id', 'u5']], on='user_id', how='left')
# 每周登录的平均时间间隔u6 按小时
matrix['u6'] = round(matrix['u5']/matrix['u1'],2) 
matrix.head()

,user_id,role_id,login_time,game_id,role_level,device_no,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6
0,112131331,149481937,2020-06-01 10:20:11,1001049,89,F6FA4ED0B437A712D882FF0A5046E183,111.52.50.96,1605-A01,6.0.1,wifi,1.0,4002000,21,1,1,4,19.46,0.93
1,101748615,141238538,2020-06-01 10:19:27,1000840,160,69A334395275B8F38C46D2BAADD31654,111.113.169.37,PAR-AL00,9,4g,3.0,4000000,142,1,2,12,20.84,0.15
2,110447841,148183494,2020-06-01 10:20:22,1000748,62,8AF169CB397754A3A9FCF671D8B0545D,183.240.10.135,Mate 10 pro,6.0.1,wifi,0.0,3003002,19,1,2,4,23.17,1.22
3,77143990,150540428,2020-06-01 10:19:39,1000571,0,7979847E86528DEBE3322875940579F9,223.88.141.90,V1838A,10,wifi,1.0,4000000,230,2,2,17,23.72,0.10
4,104448650,146453280,2020-06-01 10:20:15,1000840,121,09070BB76C84D18F07EF4F501391BE7F,120.235.35.13,V1814A,9,wifi,1.0,4000000,51,2,1,8,18.21,0.36


In [137]:
# # 按uesr_id,role_id进行聚合，查看user_id,role_id交互行为
# groups = login_info.groupby(['user_id','role_id'])
# # user_id, role_id本周登录次数 r1
# temp = groups.size().reset_index().rename(columns={0:'r1'})
# matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')
# # user_id, role_id本周登录的设备数r2
# temp = groups['device_no'].agg([('r2', 'nunique')]).reset_index()
# matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')
# # user,role_id本周使用的ip数r3
# temp = groups['ip'].agg([('r3', 'nunique')]).reset_index()
# matrix = pd.merge(matrix, temp, on=['user_id','role_id'], how='left')
# # user,role_id本周登录的平均时间间隔r4 按小时
# login_info['login_time'] = pd.to_datetime(login_info['login_time'])
# temp = groups['login_time'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
# temp['r4'] = round((temp['L_time']-temp['F_time']).dt.seconds/3600, 2)
# matrix = pd.merge(matrix, temp[['user_id','role_id', 'r4']], on=['user_id','role_id'], how='left')
# matrix['r4'] = round(matrix['r4']/matrix['r1'],2) 

In [24]:
# 按game_id进行分组聚合，查看game的交互行为
groups = login_info.groupby(['game_id'])
# 本周game的登录user_id数 g7
temp = groups['user_id'].agg([('g7', 'nunique')]).reset_index()
matrix_1 = pd.merge(matrix, temp, on='game_id', how='left')
# 本周game的登录role_id数 g8
temp = groups['role_id'].agg([('g8', 'nunique')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='game_id', how='left')
# 本周game的登录device_on数 g9
temp = groups['device_no'].agg([('g9', 'nunique')]).reset_index()
matrix_1 = pd.merge(matrix_1, temp, on='game_id', how='left')
# 本周game的登录总次数 g10
temp = groups.size().reset_index().rename(columns={0:'g10'})
matrix_1 = pd.merge(matrix_1, temp, on='game_id', how='left')
matrix_1.head()

,user_id,role_id,login_time,game_id,role_level,device_no,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6,g7,g8,g9,g10
0,112131331,149481937,2020-06-01 10:20:11,1001049,89,F6FA4ED0B437A712D882FF0A5046E183,111.52.50.96,1605-A01,6.0.1,wifi,1.0,4002000,21,1,1,4,19.46,0.93,29848,32547,28829,336843
1,101748615,141238538,2020-06-01 10:19:27,1000840,160,69A334395275B8F38C46D2BAADD31654,111.113.169.37,PAR-AL00,9,4g,3.0,4000000,142,1,2,12,20.84,0.15,136657,151468,129869,1813367
2,110447841,148183494,2020-06-01 10:20:22,1000748,62,8AF169CB397754A3A9FCF671D8B0545D,183.240.10.135,Mate 10 pro,6.0.1,wifi,0.0,3003002,19,1,2,4,23.17,1.22,7139,7891,1571,534792
3,77143990,150540428,2020-06-01 10:19:39,1000571,0,7979847E86528DEBE3322875940579F9,223.88.141.90,V1838A,10,wifi,1.0,4000000,230,2,2,17,23.72,0.10,58284,58799,5102,866637
4,104448650,146453280,2020-06-01 10:20:15,1000840,121,09070BB76C84D18F07EF4F501391BE7F,120.235.35.13,V1814A,9,wifi,1.0,4000000,51,2,1,8,18.21,0.36,136657,151468,129869,1813367


In [25]:
matrix_1['user_id'].nunique()

711528

In [26]:
matrix_1['role_id'].nunique()

809831

In [27]:
matrix_1.isna().sum()

user_id             0
role_id             0
login_time          0
game_id             0
role_level          0
device_no      182037
ip             111589
p_model        213554
sys_ver        213554
net_w         5270988
net_type       207710
sdk_no         109063
u1                  0
u2                  0
u3                  0
u4                  0
u5                  0
u6                  0
g7                  0
g8                  0
g9                  0
g10                 0
dtype: int64

In [29]:
# 根据role_id把缺失的p_model补全，众数补全

p_model_filler = pd.DataFrame(matrix_1.groupby(['role_id'])['p_model'].apply(mode_1), columns=['p_model'])
matrix_1.update(p_model_filler, overwrite=False)

sys_ver_filler = pd.DataFrame(matrix_1.groupby(['role_id'])['sys_ver'].apply(mode_1), columns=['sys_ver'])
matrix_1.update(sys_ver_filler, overwrite=False)

net_w_filler = pd.DataFrame(matrix_1.groupby(['role_id'])['net_w'].apply(mode_1), columns=['net_w'])
matrix_1.update(net_w_filler, overwrite=False)

net_w_filler = pd.DataFrame(matrix_1.groupby(['role_id'])['net_w'].apply(mode_1), columns=['net_w'])
matrix_1.update(net_w_filler, overwrite=False)

sdk_no_filler = pd.DataFrame(matrix_1.groupby(['role_id'])['sdk_no'].apply(mode_1), columns=['sdk_no'])
matrix_1.update(sdk_no_filler, overwrite=False)

In [30]:
matrix_1.sort_values(by='login_time')
matrix_1 = matrix_1.drop_duplicates('role_id', keep='last')

In [31]:
del login_info, matrix
gc.collect()

40

In [32]:
matrix_1.to_csv('./FE_data/matrix_login.csv', index=None)
matrix_1.shape

(809831, 22)

# 3 处理role_info

In [22]:
role_info = pd.read_csv('./role_info.csv')

C:\Users\Administrator\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
role_info.drop_duplicates('role_id', keep='first', inplace=True)
print(role_info.shape)
role_info.head()

(56039990, 4)


,role_id,is_create_role,is_create_no,create_role_time
0,131669783.0,1.0,0,2019-08-22 00:00:00
1,131669784.0,1.0,1,2019-08-22 00:00:00
2,131669785.0,1.0,1,2019-08-22 00:00:01
3,131669786.0,1.0,0,2019-08-22 00:00:02
4,131669788.0,1.0,1,2019-08-22 00:00:03


In [24]:
role_info['is_create_role'].value_counts()

1.0    47893041
0.0     4123129
Name: is_create_role, dtype: int64

In [25]:
role_info['is_create_no'] = pd.to_numeric(role_info['is_create_no'], errors='coerce')
role_info['create_role_time'] = pd.to_datetime(role_info['create_role_time'])

In [26]:
role_info['is_create_no'].value_counts()

1.0    38827382
0.0     8476425
Name: is_create_no, dtype: int64

In [65]:
role_info.to_csv('./FE_data/role_info.csv', index=None)

# 4 处理users_info

In [148]:
users_info = pd.read_csv('users_info.csv')
users_info_dim = pd.read_csv('users_info_dim.csv')

C:\Users\Administrator\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [149]:
# 压缩数据
users_info = reduce_mem_usage(users_info)
users_info_dim = reduce_mem_usage(users_info_dim)

Mem. usage decreased to 2254.64 Mb (45.3% reduction)
Mem. usage decreased to 90.60 Mb (70.8% reduction)


In [150]:
users_info.head()

,user_id,platform,user_type,user_flag,mobile,game_id,device_no,reg_date
0,1,1,1,1,1.37169e+10,1,0C442360BC70203E6FAD614717E9AE7F,2014-10-29 19:23:00
1,2,1,1,1,1.39757e+10,1,79033D7C17B1A8BA8F80434056E18110,2014-10-29 21:48:35
2,5,1,1,1,NaN,1,C46222621B7E0040E71C331DA9D436A9,2014-10-30 12:06:04
3,7,1,1,1,NaN,1,F8ED9D8D4CE72673527ED66A186652C2,2014-10-31 19:11:22
4,10,1,1,1,NaN,3,D59220AE6B1B1D23B32775BF1C1F540C,2014-11-03 13:52:16


In [151]:
users_info['mobile'] = pd.to_numeric(users_info['mobile'],errors='coerce')

In [152]:
# 标记用户时否绑定了手机，绑定;1    未绑定：0
users_info['mobile'] = users_info['mobile'].apply(lambda x:0 if np.isnan(x) == True else 1)

In [153]:
users_info['mobile'].value_counts()

0    59667285
1     7880104
Name: mobile, dtype: int64

In [154]:
users_info_dim.shape

(13571632, 3)

In [155]:
users_info.shape

(67547389, 8)

In [156]:
users_info_dim.head()

,user_id,game_pay_num,chmoney_pay_num
0,90016683,0,0
1,89985846,0,0
2,89973614,0,0
3,89967538,0,0
4,89975343,0,0


In [157]:
users_info_dim['game_pay_num'].value_counts()

0      13076350
1        337431
2         79925
3         28502
4         17574
         ...   
69            1
68            1
37            1
229           1
231           1
Name: game_pay_num, Length: 102, dtype: int64

In [158]:
# 合并user_info, user_info_dim 注 user_info_dim只有中间件用户的数据
users_info_merge = pd.merge(users_info, users_info_dim, on='user_id', how='left')

In [159]:
# 判断是否滚服  注cp_role_info只有中间件用户的数据
cp_role_info = pd.read_csv('./cp_role_info.csv')
cp_role_info = cp_role_info.dropna(subset=['cp_role_id'])
groups = cp_role_info.groupby('cp_role_id')
temp = groups['server_id'].agg([('server_sum', 'nunique')]).reset_index()
cp_role_info = pd.merge(cp_role_info, temp, on='cp_role_id', how='left')
cp_role_info['is_gunfu'] = cp_role_info['server_sum'].apply(lambda x:0 if x==1 else 1)
cp_role_info.head()

,cp_role_id,game_id,user_id,server_id,server_sum,is_gunfu
0,8205610009986,125,13154637,30268,1,0
1,8200330038526,125,90016683,21552,1,0
2,8206870001263,125,4531551,1013823,1,0
3,8200330038525,125,90004615,21552,1,0
4,8206920001264,125,6659757,1015527,1,0


In [160]:
cp_role_info['user_id'].nunique()

13381301

In [161]:
cp_role_info['is_gunfu'].value_counts()

0    13883079
1     1058343
Name: is_gunfu, dtype: int64

In [162]:
# 按全量user_id进行合并
users_info_all = pd.merge(users_info_merge, cp_role_info[['user_id', 'server_sum', 'is_gunfu']], on='user_id', how='left')

In [163]:
users_info_all.head()

,user_id,platform,user_type,user_flag,mobile,game_id,device_no,reg_date,game_pay_num,chmoney_pay_num,server_sum,is_gunfu
0,1,1,1,1,1,1,0C442360BC70203E6FAD614717E9AE7F,2014-10-29 19:23:00,NaN,NaN,NaN,NaN
1,2,1,1,1,1,1,79033D7C17B1A8BA8F80434056E18110,2014-10-29 21:48:35,NaN,NaN,NaN,NaN
2,5,1,1,1,0,1,C46222621B7E0040E71C331DA9D436A9,2014-10-30 12:06:04,NaN,NaN,NaN,NaN
3,7,1,1,1,0,1,F8ED9D8D4CE72673527ED66A186652C2,2014-10-31 19:11:22,NaN,NaN,NaN,NaN
4,10,1,1,1,0,3,D59220AE6B1B1D23B32775BF1C1F540C,2014-11-03 13:52:16,NaN,NaN,NaN,NaN


In [164]:
# user_id玩过的game数  m1
groups = users_info_all.groupby('user_id')
temp = groups['game_id'].agg([('m1', 'nunique')]).reset_index()
matrix_2 = pd.merge(users_info_all, temp, on='user_id', how='left')
# game的user_id人数 m2
groups = users_info_all.groupby('game_id')
temp = groups['user_id'].agg([('m2', 'nunique')]).reset_index()
matrix_2 = pd.merge(matrix_2, temp, on='game_id', how='left')
matrix_2.head()

,user_id,platform,user_type,user_flag,mobile,game_id,device_no,reg_date,game_pay_num,chmoney_pay_num,server_sum,is_gunfu,m1,m2
0,1,1,1,1,1,1,0C442360BC70203E6FAD614717E9AE7F,2014-10-29 19:23:00,NaN,NaN,NaN,NaN,1,5610
1,2,1,1,1,1,1,79033D7C17B1A8BA8F80434056E18110,2014-10-29 21:48:35,NaN,NaN,NaN,NaN,1,5610
2,5,1,1,1,0,1,C46222621B7E0040E71C331DA9D436A9,2014-10-30 12:06:04,NaN,NaN,NaN,NaN,1,5610
3,7,1,1,1,0,1,F8ED9D8D4CE72673527ED66A186652C2,2014-10-31 19:11:22,NaN,NaN,NaN,NaN,1,5610
4,10,1,1,1,0,3,D59220AE6B1B1D23B32775BF1C1F540C,2014-11-03 13:52:16,NaN,NaN,NaN,NaN,1,301458


In [165]:
matrix_2.shape

(69106874, 14)

In [166]:
matrix_2['user_id'].nunique()

67547389

In [167]:
matrix_2.to_csv('./FE_data/matrix_user.csv', index=None)

In [168]:
del users_info, users_info_dim, users_info_merge, cp_role_info, users_info_all
gc.collect()

80

# 合并表

In [90]:
print(matrix_1.shape)
print(role_info.shape)

(711528, 22)
(56039990, 4)


In [91]:
matrix_1.head()

,user_id,role_id,login_time,game_id,role_level,device_no,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6,g7,g8,g9,g10
48,114121797,151077491,2020-06-01 10:20:29,1000840,4,4CF651668A8E5E7349F06B7234880D6E,183.39.33.116,PCAM10,9,4g,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367
103,114121784,151077479,2020-06-01 10:20:04,1000840,4,D7E5BB5701CDBB767508AB8387CB47A0,183.199.71.108,HRY-AL00a,10,wifi,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367
224,114121758,151077468,2020-06-01 10:19:38,1001184,0,7F9E7F86AE8A023A1355AA88A7311AF5,39.144.4.25,HRY-AL00a,10,4g,0.0,4005000,1,1,1,1,0.0,0.0,2799,2883,2463,20322
351,108850615,146988322,2020-06-01 10:21:07,1001040,73,876A39E0410F339D7ACAA8EE77273818,42.249.39.229,OPPO A59s,5.1,NaN,1.0,4005000,1,1,1,1,0.0,0.0,9564,10386,9338,222150
591,93006496,133843444,2020-06-01 10:20:52,1000769,226,A1721D13D0CDA7A82610DF052EB3D7F5,61.158.146.219,VOG-AL00,10,4g,2.0,3003001,1,1,1,1,0.0,0.0,17669,19404,12415,429231


In [92]:
role_info.head()

,role_id,is_create_role,is_create_no,create_role_time
0,131669783.0,1.0,0.0,2019-08-22 00:00:00
1,131669784.0,1.0,1.0,2019-08-22 00:00:00
2,131669785.0,1.0,1.0,2019-08-22 00:00:01
3,131669786.0,1.0,0.0,2019-08-22 00:00:02
4,131669788.0,1.0,1.0,2019-08-22 00:00:03


In [96]:
# 合并登录信息login_info(matrix_1)与角色信息role_info
df_login_role = pd.merge(matrix_1, role_info, on='role_id', how='left')
print(df_login_role.shape)

(711528, 25)


In [97]:
df_login_role.head()

,user_id,role_id,login_time,game_id,role_level,device_no,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6,g7,g8,g9,g10,is_create_role,is_create_no,create_role_time
0,114121797,151077491,2020-06-01 10:20:29,1000840,4,4CF651668A8E5E7349F06B7234880D6E,183.39.33.116,PCAM10,9,4g,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,2020-06-01 10:20:29
1,114121784,151077479,2020-06-01 10:20:04,1000840,4,D7E5BB5701CDBB767508AB8387CB47A0,183.199.71.108,HRY-AL00a,10,wifi,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,2020-06-01 10:20:04
2,114121758,151077468,2020-06-01 10:19:38,1001184,0,7F9E7F86AE8A023A1355AA88A7311AF5,39.144.4.25,HRY-AL00a,10,4g,0.0,4005000,1,1,1,1,0.0,0.0,2799,2883,2463,20322,1.0,1.0,2020-06-01 10:19:38
3,108850615,146988322,2020-06-01 10:21:07,1001040,73,876A39E0410F339D7ACAA8EE77273818,42.249.39.229,OPPO A59s,5.1,NaN,1.0,4005000,1,1,1,1,0.0,0.0,9564,10386,9338,222150,1.0,1.0,2020-03-28 16:32:55
4,93006496,133843444,2020-06-01 10:20:52,1000769,226,A1721D13D0CDA7A82610DF052EB3D7F5,61.158.146.219,VOG-AL00,10,4g,2.0,3003001,1,1,1,1,0.0,0.0,17669,19404,12415,429231,1.0,1.0,2019-09-09 12:19:03


In [51]:
# 合并df_login_role与角色信息user_info(matrix_2)
matrix_2.drop(['device_no'], axis=1, inplace=True)
matrix_2.head()

,user_id,platform,user_type,user_flag,mobile,game_id,reg_date,game_pay_num,chmoney_pay_num,server_sum,is_gunfu,m1,m2
0,1,1,1,1,1,1,2014-10-29 19:23:00,NaN,NaN,NaN,NaN,1,5610
1,2,1,1,1,1,1,2014-10-29 21:48:35,NaN,NaN,NaN,NaN,1,5610
2,5,1,1,1,0,1,2014-10-30 12:06:04,NaN,NaN,NaN,NaN,1,5610
3,7,1,1,1,0,1,2014-10-31 19:11:22,NaN,NaN,NaN,NaN,1,5610
4,10,1,1,1,0,3,2014-11-03 13:52:16,NaN,NaN,NaN,NaN,1,301458


In [76]:
matrix_2['user_id'].nunique()

67547389

In [106]:
df_login_role_users = pd.merge(df_login_role, matrix_2, on=['user_id', 'game_id'], how='left')

In [105]:

gc.collect()

54397

In [107]:
print(df_login_role_users.shape)
print(df_login_role.shape)
print(matrix_2.shape)

(1025224, 36)
(711528, 25)
(69106874, 13)


In [108]:
df_login_role_users.head()

,user_id,role_id,login_time,game_id,role_level,device_no,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6,g7,g8,g9,g10,is_create_role,is_create_no,create_role_time,platform,user_type,user_flag,mobile,reg_date,game_pay_num,chmoney_pay_num,server_sum,is_gunfu,m1,m2
0,114121797,151077491,2020-06-01 10:20:29,1000840,4,4CF651668A8E5E7349F06B7234880D6E,183.39.33.116,PCAM10,9,4g,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,2020-06-01 10:20:29,1.0,1.0,1.0,0.0,2020-06-01 10:20:25,0.0,0.0,1.0,0.0,1.0,2815759.0
1,114121784,151077479,2020-06-01 10:20:04,1000840,4,D7E5BB5701CDBB767508AB8387CB47A0,183.199.71.108,HRY-AL00a,10,wifi,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,2020-06-01 10:20:04,1.0,1.0,1.0,0.0,2020-06-01 10:19:59,0.0,0.0,1.0,0.0,1.0,2815759.0
2,114121758,151077468,2020-06-01 10:19:38,1001184,0,7F9E7F86AE8A023A1355AA88A7311AF5,39.144.4.25,HRY-AL00a,10,4g,0.0,4005000,1,1,1,1,0.0,0.0,2799,2883,2463,20322,1.0,1.0,2020-06-01 10:19:38,1.0,1.0,1.0,0.0,2020-06-01 10:19:14,0.0,0.0,1.0,0.0,1.0,32322.0
3,108850615,146988322,2020-06-01 10:21:07,1001040,73,876A39E0410F339D7ACAA8EE77273818,42.249.39.229,OPPO A59s,5.1,NaN,1.0,4005000,1,1,1,1,0.0,0.0,9564,10386,9338,222150,1.0,1.0,2020-03-28 16:32:55,1.0,2.0,1.0,1.0,2020-03-28 16:32:47,0.0,0.0,1.0,0.0,1.0,408350.0
4,93006496,133843444,2020-06-01 10:20:52,1000769,226,A1721D13D0CDA7A82610DF052EB3D7F5,61.158.146.219,VOG-AL00,10,4g,2.0,3003001,1,1,1,1,0.0,0.0,17669,19404,12415,429231,1.0,1.0,2019-09-09 12:19:03,1.0,1.0,1.0,0.0,2019-09-09 12:17:56,3.0,0.0,1.0,0.0,1.0,1577912.0


In [110]:
df_login_role_users.sort_values(by='login_time')
df_login_role_users = df_login_role_users.drop_duplicates(subset='user_id', keep='last')

In [113]:
# 合并df_login_role_users与角色信息product_info
print(matrix_product.shape)

(202862, 34)


In [112]:
matrix_product.head()

,order_no,game_id,channel_id,source_id,server_id,role_id,user_id,pay_type,money,give_money,rmb_money,ch_money,coupon_money,coupon_code,coupon_type,pay_count,role_pay_count,role_level,device_no,pay_time,post_time,active_pay_count,dt,is_pay,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
0,H20200607235949599113164010953,1001081,20814,307611,30860,13273385,113164010,27,1.0,0.0,1.0,0.0,0.0,NaN,0,10,10,128,F17735C4-5BBE-4E48-9BF8-6F849DDBF263,2020-06-08 00:00:09,2020-06-08 00:00:09,10,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0,196,301,594
1,H20200608000012775112767396777,1001081,20938,306794,30582,13181677,112767396,27,1.0,0.0,1.0,0.0,0.0,NaN,0,52,52,163,6A05A10F-CB14-4837-A4EE-B67261C65768,2020-06-08 00:00:25,2020-06-08 00:00:25,52,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0,196,301,594
2,H20200608000015629110588172745,1000952,20789,292133,30908,13288577,110588172,2,1.0,0.0,1.0,0.0,0.0,NaN,0,70,50,0,08D275B5-A684-453B-BE04-CD9D36A84473,2020-06-08 00:00:25,2020-06-08 00:00:25,70,2020-06-08,1,12410,1662,1684,1622,77.028445,1.787326,0.0,210,271,801
3,H2020060800001822598005731142,1000952,20769,227280,22646,10183475,98005731,27,6.0,0.0,6.0,0.0,0.0,NaN,0,106,106,0,19D6756F-6FFD-454A-A607-A40845F26DF9,2020-06-08 00:00:32,2020-06-08 00:00:32,62,2020-06-08,1,12410,1662,1684,1622,77.028445,1.787326,0.0,210,271,801
4,H20200608000021856104706588285,1001081,20774,272572,26286,11690378,104706588,2,3.0,0.0,3.0,0.0,0.0,NaN,0,172,152,235,7718269B-BEA5-4143-A5AB-EE0E414E4B03,2020-06-08 00:00:45,2020-06-08 00:00:45,152,2020-06-08,1,11396,1814,1843,1738,63.734468,1.234695,0.0,196,301,594


In [114]:
matrix_product.columns

Index(['order_no', 'game_id', 'channel_id', 'source_id', 'server_id',
       'role_id', 'user_id', 'pay_type', 'money', 'give_money', 'rmb_money',
       'ch_money', 'coupon_money', 'coupon_code', 'coupon_type', 'pay_count',
       'role_pay_count', 'role_level', 'device_no', 'pay_time', 'post_time',
       'active_pay_count', 'dt', 'is_pay', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6',
       'p7', 'p8', 'p9', 'p10'],
      dtype='object')

In [129]:
# 合并游戏game特征p1-p10
game_feature = matrix_product[['game_id', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6',
       'p7', 'p8', 'p9', 'p10']]
game_feature = game_feature.drop_duplicates('game_id')
df = pd.merge(df_login_role_users, game_feature, on='game_id', how='left')

In [130]:
# 合并user_id 是否付费
user_pay = matrix_product[['user_id', 'is_pay']]
user_pay = user_pay.drop_duplicates('user_id')
df = pd.merge(df, user_pay, on='user_id', how='left')

In [136]:
df.columns

Index(['user_id', 'role_id', 'login_time', 'game_id', 'role_level',
       'device_no', 'ip', 'p_model', 'sys_ver', 'net_w', 'net_type', 'sdk_no',
       'u1', 'u2', 'u3', 'u4', 'u5', 'u6', 'g7', 'g8', 'g9', 'g10',
       'is_create_role', 'is_create_no', 'create_role_time', 'platform',
       'user_type', 'user_flag', 'mobile', 'reg_date', 'game_pay_num',
       'chmoney_pay_num', 'server_sum', 'is_gunfu', 'm1', 'm2', 'p1', 'p2',
       'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'is_pay'],
      dtype='object')

In [131]:
df.head()

,user_id,role_id,login_time,game_id,role_level,device_no,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6,g7,g8,g9,g10,is_create_role,is_create_no,create_role_time,platform,user_type,user_flag,mobile,reg_date,game_pay_num,chmoney_pay_num,server_sum,is_gunfu,m1,m2,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,is_pay
0,114121797,151077491,2020-06-01 10:20:29,1000840,4,4CF651668A8E5E7349F06B7234880D6E,183.39.33.116,PCAM10,9,4g,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,2020-06-01 10:20:29,1.0,1.0,1.0,0.0,2020-06-01 10:20:25,0.0,0.0,1.0,0.0,1.0,2815759.0,16429.0,5270.0,5413.0,5206.0,133.462657,0.770520,0.027391,1021.0,1209.0,1033.0,NaN
1,114121784,151077479,2020-06-01 10:20:04,1000840,4,D7E5BB5701CDBB767508AB8387CB47A0,183.199.71.108,HRY-AL00a,10,wifi,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,2020-06-01 10:20:04,1.0,1.0,1.0,0.0,2020-06-01 10:19:59,0.0,0.0,1.0,0.0,1.0,2815759.0,16429.0,5270.0,5413.0,5206.0,133.462657,0.770520,0.027391,1021.0,1209.0,1033.0,NaN
2,114121758,151077468,2020-06-01 10:19:38,1001184,0,7F9E7F86AE8A023A1355AA88A7311AF5,39.144.4.25,HRY-AL00a,10,4g,0.0,4005000,1,1,1,1,0.0,0.0,2799,2883,2463,20322,1.0,1.0,2020-06-01 10:19:38,1.0,1.0,1.0,0.0,2020-06-01 10:19:14,0.0,0.0,1.0,0.0,1.0,32322.0,240.0,90.0,92.0,85.0,34.900000,0.029500,0.000000,45.0,55.0,49.0,NaN
3,108850615,146988322,2020-06-01 10:21:07,1001040,73,876A39E0410F339D7ACAA8EE77273818,42.249.39.229,OPPO A59s,5.1,NaN,1.0,4005000,1,1,1,1,0.0,0.0,9564,10386,9338,222150,1.0,1.0,2020-03-28 16:32:55,1.0,2.0,1.0,1.0,2020-03-28 16:32:47,0.0,0.0,1.0,0.0,1.0,408350.0,5562.0,723.0,737.0,717.0,60.310859,0.528357,0.191837,117.0,151.0,123.0,NaN
4,93006496,133843444,2020-06-01 10:20:52,1000769,226,A1721D13D0CDA7A82610DF052EB3D7F5,61.158.146.219,VOG-AL00,10,4g,2.0,3003001,1,1,1,1,0.0,0.0,17669,19404,12415,429231,1.0,1.0,2019-09-09 12:19:03,1.0,1.0,1.0,0.0,2019-09-09 12:17:56,3.0,0.0,1.0,0.0,1.0,1577912.0,3730.0,760.0,772.0,680.0,64.489008,3.191399,0.060858,196.0,260.0,162.0,NaN


In [135]:
df['is_pay'] = df['is_pay'].fillna(0)
df['is_pay'].value_counts()

0.0    674594
1.0     36934
Name: is_pay, dtype: int64

In [141]:
data = df.drop(['role_id', 'login_time','game_id','device_no','create_role_time','reg_date'], axis=1)

In [146]:
data = data.rename(columns={'is_gunfu':'rollsrv'})
# game本周的总付费次数/总登录次数s1
# game本周的付费user数/登录user数s2
# game本周的付费role数/登录role数s3
# game本周的付费device数/登录denvice次数s4
# game本周的新增付费user数/付费user数s5
# game本周的新增付费role数/付费role数s6
# game本周的新增付费device数/付费device数s7
data['s1'] = data['p1']/data['g10']
data['s2'] = data['p2']/data['g7']
data['s3'] = data['p3']/data['g8']
data['s4'] = data['p4']/data['g9']
data['s5'] = data['p8']/data['p2']
data['s6'] = data['p9']/data['p3']
data['s7'] = data['p10']/data['p4']

In [150]:
data.head()

,user_id,role_level,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6,g7,g8,g9,g10,is_create_role,is_create_no,platform,user_type,user_flag,mobile,game_pay_num,chmoney_pay_num,server_sum,rollsrv,m1,m2,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,is_pay,s1,s2,s3,s4,s5,s6,s7
0,114121797,4,183.39.33.116,PCAM10,9,4g,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2815759.0,16429.0,5270.0,5413.0,5206.0,133.462657,0.770520,0.027391,1021.0,1209.0,1033.0,0.0,0.009060,0.038564,0.035737,0.040087,0.193738,0.223351,0.198425
1,114121784,4,183.199.71.108,HRY-AL00a,10,wifi,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2815759.0,16429.0,5270.0,5413.0,5206.0,133.462657,0.770520,0.027391,1021.0,1209.0,1033.0,0.0,0.009060,0.038564,0.035737,0.040087,0.193738,0.223351,0.198425
2,114121758,0,39.144.4.25,HRY-AL00a,10,4g,0.0,4005000,1,1,1,1,0.0,0.0,2799,2883,2463,20322,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,32322.0,240.0,90.0,92.0,85.0,34.900000,0.029500,0.000000,45.0,55.0,49.0,0.0,0.011810,0.032154,0.031911,0.034511,0.500000,0.597826,0.576471
3,108850615,73,42.249.39.229,OPPO A59s,5.1,NaN,1.0,4005000,1,1,1,1,0.0,0.0,9564,10386,9338,222150,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,408350.0,5562.0,723.0,737.0,717.0,60.310859,0.528357,0.191837,117.0,151.0,123.0,0.0,0.025037,0.075596,0.070961,0.076783,0.161826,0.204885,0.171548
4,93006496,226,61.158.146.219,VOG-AL00,10,4g,2.0,3003001,1,1,1,1,0.0,0.0,17669,19404,12415,429231,1.0,1.0,1.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,1.0,1577912.0,3730.0,760.0,772.0,680.0,64.489008,3.191399,0.060858,196.0,260.0,162.0,0.0,0.008690,0.043013,0.039786,0.054772,0.257895,0.336788,0.238235


In [155]:
data.shape

(711528, 48)

In [190]:
data['ip'].value_counts().iloc[0:500]

180.126.81.98     9504
27.203.33.171     8248
49.71.12.197      1732
183.236.48.92     1299
49.86.108.251     1297
                  ... 
113.96.65.113       30
120.244.226.20      30
223.80.226.246      30
27.17.195.221       30
124.64.16.180       30
Name: ip, Length: 500, dtype: int64

In [191]:
ip_df = data[data['ip'].isin(data['ip'].value_counts().iloc[0:500].index)]
data.drop(ip_df.index, axis=0, inplace=True)
data.head()

,user_id,role_level,ip,p_model,sys_ver,net_w,net_type,sdk_no,u1,u2,u3,u4,u5,u6,g7,g8,g9,g10,is_create_role,is_create_no,platform,user_type,user_flag,mobile,game_pay_num,chmoney_pay_num,server_sum,rollsrv,m1,m2,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,is_pay,s1,s2,s3,s4,s5,s6,s7
0,114121797,4,183.39.33.116,PCAM10,9,4g,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2815759.0,16429.0,5270.0,5413.0,5206.0,133.462657,0.770520,0.027391,1021.0,1209.0,1033.0,0.0,0.009060,0.038564,0.035737,0.040087,0.193738,0.223351,0.198425
1,114121784,4,183.199.71.108,HRY-AL00a,10,wifi,0.0,4005000,1,1,1,1,0.0,0.0,136657,151468,129869,1813367,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2815759.0,16429.0,5270.0,5413.0,5206.0,133.462657,0.770520,0.027391,1021.0,1209.0,1033.0,0.0,0.009060,0.038564,0.035737,0.040087,0.193738,0.223351,0.198425
2,114121758,0,39.144.4.25,HRY-AL00a,10,4g,0.0,4005000,1,1,1,1,0.0,0.0,2799,2883,2463,20322,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,32322.0,240.0,90.0,92.0,85.0,34.900000,0.029500,0.000000,45.0,55.0,49.0,0.0,0.011810,0.032154,0.031911,0.034511,0.500000,0.597826,0.576471
3,108850615,73,42.249.39.229,OPPO A59s,5.1,NaN,1.0,4005000,1,1,1,1,0.0,0.0,9564,10386,9338,222150,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,408350.0,5562.0,723.0,737.0,717.0,60.310859,0.528357,0.191837,117.0,151.0,123.0,0.0,0.025037,0.075596,0.070961,0.076783,0.161826,0.204885,0.171548
4,93006496,226,61.158.146.219,VOG-AL00,10,4g,2.0,3003001,1,1,1,1,0.0,0.0,17669,19404,12415,429231,1.0,1.0,1.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,1.0,1577912.0,3730.0,760.0,772.0,680.0,64.489008,3.191399,0.060858,196.0,260.0,162.0,0.0,0.008690,0.043013,0.039786,0.054772,0.257895,0.336788,0.238235


In [ ]:
import geoip2.database
def getLoc(ip):
    # 传入下载的数据的地址
    reader = geoip2.database.Reader("./GeoLite2-City.mmdb")
    data = reader.city(ip)
    # print("ip地址：", ip)
    # names['zh-CN']即可转换为中文
    # print("国家: ", data.country.names['zh-CN'])  
    # print("省份: ", data.subdivisions.most_specific.names['zh-CN'])
    # print("城市: ", data.city.names['zh-CN'])
    # print("纬度: ", data.location.latitude)
    # print("经度: ", data.location.longitude)
    result = data.city.names['zh-CN']
    return result

In [198]:
city_list = []
for ip in data['ip']:
    try:
        getLoc(str(ip))
    except KeyError: city=np.nan
    except ValueError:city=np.nan
    except BaseException:city=np.nan
    else:
        city = getLoc(str(ip))
    city_list.append(city)
    

In [201]:
data['city'] = city_list

In [203]:
data['city'].isna().sum()

130090

In [204]:
data.to_csv('./data.csv', index=False)